In [1]:
import warnings
warnings.filterwarnings(action='ignore') #경고 메시지 무시

In [19]:
import pandas as pd
import geopandas as gpd
import fiona

In [22]:
import os
os.getcwd()

'c:\\Users\\soohan\\data-visualization\\plotly'

In [23]:
# CCTV CSV 로딩
df_cctv = pd.read_csv('../dataset/전국CCTV표준데이터.csv', index_col=None, header=0, 
                 names=['admin', 'addr1', 'addr2', 'usetype', 'cnt', 'pixel', 'azimuth', 'keep', 'inst_date', 'tel', 'lat', 'lon', 'up_date', 'offer_cd', 'offer', 'id2'], 
                 usecols=['admin', 'addr1', 'lat', 'lon'], 
                 dtype={'admin':object, 'addr1':object, 'lat':float, 'lon':float},
                 encoding="EUC-KR"    )
df_cctv.head()

ParserError: Too many columns specified: expected 16 and found 15

In [21]:
df_cctv

NameError: name 'df_cctv' is not defined